# Échantillonnage direct de l'espace des motifs

### BRUNEAU Richard - VASLIN Pierre





In [2]:
import numpy as np
import scipy.special as sps
import math
import matplotlib
import pandas as pd
import random

class DataSet():
    def __init__(self,df:pd.DataFrame):
        self.df = df
        self.sizes = np.zeros(self.df.shape[0],dtype=int)
        for i in range(self.df.shape[0]):
            self.sizes[i] = self.df.iloc[i].count()
        # Sizes est pour connaitre la taille d'une ligne en o(1)
        # Pandas gère mal la variation du nombre de colonne dans une ligne dans un dataFrame, par concéquent 
        # il recalcul à chaque fois le nombre d'élément non null o(n)


In [3]:
# Le dataframe pour les tests
df = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/chess.txt", sep=" ",header=None)
del df[37]
ds = DataSet(df)
ds.df.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,1,3,5,7,9,11,13,15,17,19,...,56,58,60,62,64,66,68,70,72,74
1,1,3,5,7,9,12,13,15,17,19,...,56,58,60,62,64,66,68,70,72,74
2,1,3,5,7,9,12,13,16,17,19,...,56,58,60,62,64,66,68,70,72,74
3,1,3,5,7,9,11,13,15,17,20,...,56,58,60,62,64,66,68,70,72,74
4,1,3,5,7,9,11,13,15,17,19,...,56,58,60,62,64,66,68,70,72,74


## Question 1

In [4]:
def algoFrequences(ds:DataSet,nb_pattern)-> tuple:
  df = ds.df
  sizes = ds.sizes
  R = []
  IsInR = set()
  P: dict = dict()
  w = np.zeros(df.shape[0])
  totalW = 0
    
  # set les probas
  for i in range(len(w)):
    w[i] = math.pow(2,sizes[i])
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    random_row = random.uniform(0,totalW)
    # On cherche la ligne
    row = 0
    v = 0
    for i in range(len(w)):
      if  v > random_row:
        row = i - 1
        break
      v += w[i]
    # On selectionne un motif 
    pattern = np.array(df.iloc[row][:sizes[row]])
    random_v = random.randint(1, len(pattern) - 1 )
    for i in range(len(pattern)- random_v):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    # On ajoute seulement les motifs non présent dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R

In [5]:
algoFrequences(ds,10)

[array([11, 15, 17, 19, 27, 34, 36, 40, 46, 66, 69, 70], dtype=int64),
 array([ 1,  3,  6,  7, 14, 24, 25, 27, 40, 43, 46, 50, 52, 56, 58, 60, 62,
        65, 66, 73, 74], dtype=int64),
 array([ 3,  9, 12, 21, 27, 31, 40, 56, 64], dtype=int64),
 array([19, 27, 48], dtype=int64),
 array([ 2,  3,  7,  9, 12, 14, 16, 17, 20, 21, 23, 25, 27, 29, 31, 34, 36,
        38, 40, 42, 44, 47, 48, 51, 52, 55, 56, 58, 60, 62, 64, 66, 69, 70,
        72, 75], dtype=int64),
 array([ 3,  5,  9, 11, 13, 15, 17, 19, 21, 24, 25, 27, 29, 31, 40, 43, 46,
        48, 51, 52, 54, 57, 58, 60, 62, 65, 66, 69, 70, 73, 74],
       dtype=int64),
 array([ 2,  5,  8,  9, 11, 14, 16, 17, 21, 25, 33, 34, 40, 51, 52, 54, 57,
        60, 62, 69, 71, 72], dtype=int64),
 array([ 3,  7, 15, 62, 70], dtype=int64),
 array([ 2,  5,  7,  9, 16, 20, 21, 23, 25, 34, 36, 38, 40, 42, 44, 47, 48,
        52, 55, 56, 58, 62, 64, 69, 75], dtype=int64),
 array([ 1,  3,  5, 14, 21, 34, 38, 40, 48, 64], dtype=int64)]

## Question 2

In [6]:
def algoArea(ds:DataSet,nb_pattern)-> tuple:
  df = ds.df
  sizes = ds.sizes
  R = []
  IsInR = set()
  
  # set les probas
  w = np.zeros(df[0].count(),dtype=np.ulonglong)
  totalW:np.ulonglong= 0
  for i in range(1, len(w)):
    w[i] = np.multiply(sizes[i], np.power(2, (sizes[i] - 1)))
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    # On cherche la ligne
    random_row = random.uniform(0,totalW)
    v, row = 0,0
    for i in range(len(w)):
      if v > random_row:
        row = i - 1
        break
      v += w[i]
      
    # On set les probabilités de k (taille du motif)
    # On souhaite que le sous-ensemble est une taille calculée 
    # proportionnellement avec les tailles des datarecords (lignes) 
    ks = np.zeros(int(sizes[row]))
    totalK = 0
    for i in range(len(ks)):
        ks[i] = sps.binom(len(ks), i + 1)
        totalK += ks[i]
    #totalK = (len(ks) * (len(ks)+1))/2
    random_kp = random.uniform(0, totalK)
    # On cherche la ligne
    k, v = 0,0
    for i in range(len(ks)):
      if v > random_kp:
        k = i - 1
        break
      v += ks[i]
    
    pattern = np.array(df.iloc[row][:sizes[row]])
    for i in range(len(pattern)- k):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    
    # On ajoute seulement les motifs non présent dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R

In [7]:
algoArea(ds,10)

[array([ 5,  9, 13, 17, 19, 21, 23, 42, 44, 50, 54, 60, 64, 70],
       dtype=int64),
 array([ 5,  9, 11, 15, 17, 27, 31, 34, 40, 42, 44, 50, 52, 54, 62, 64],
       dtype=int64),
 array([ 1,  3,  7,  9, 19, 23, 29, 31, 40, 42, 44, 46, 52, 56, 58],
       dtype=int64),
 array([ 1,  3,  7,  9, 13, 25, 29, 31, 34, 36, 38, 42, 48, 54, 58, 60, 62,
        64, 68, 72, 74], dtype=int64),
 array([ 1,  7,  9, 17, 19, 21, 23, 25, 31, 34, 36, 38, 42, 52, 56, 60, 62,
        64, 68, 72], dtype=int64),
 array([ 5,  9, 15, 17, 21, 23, 27, 40, 42, 46, 50, 54, 60, 62, 66, 68, 70,
        74], dtype=int64),
 array([ 1,  5,  7,  9, 11, 23, 29, 48, 52, 58, 62, 70, 74], dtype=int64),
 array([ 9, 15, 27, 29, 31, 34, 44, 46, 50, 52, 62, 66, 68, 70, 72, 74],
       dtype=int64),
 array([ 3,  5,  7, 15, 23, 27, 29, 31, 34, 36, 40, 42, 48, 52, 54, 56, 58,
        60, 62, 64, 70], dtype=int64),
 array([ 1,  5,  7, 15, 17, 19, 25, 27, 31, 40, 42, 44, 56, 60, 68, 72],
       dtype=int64)]

## Question 3

In [8]:
def frequences(ds, patterns):
  df = ds.df
  sizes = ds.sizes
  frequencesP = np.zeros(len(patterns),dtype=float)
  indexs = np.zeros(len(patterns),dtype=int)
  lenPat = len(patterns)
  for i in range(df.shape[0]):
    indexs[True] = 0
    for j in range(ds.sizes[i]):
      for ip in range(lenPat):
        if (len(patterns[ip])!= indexs[ip] and 
            df.iloc[i][j] == patterns[ip][indexs[ip]]):
          indexs[ip] += 1
          if len(patterns[ip]) == indexs[ip]:
            frequencesP[ip] += 1.
  for i in range(len(frequencesP)):
    frequencesP[i] = (frequencesP[i])/float(df.shape[0])
  return frequencesP

In [9]:
patterns = algoFrequences(ds,10)

In [10]:
frequences(ds,patterns)

array([0.00125156, 0.00093867, 0.00031289, 0.00062578, 0.00031289,
       0.00125156, 0.00062578, 0.00031289, 0.00031289, 0.02033792])

In [11]:
def aire(ds, patterns):
  df = ds.df
  aireP = np.zeros(len(patterns),dtype=int)
  indexs = np.zeros(len(patterns),dtype=int)
  lenPat = len(patterns)
  for i in range(df.shape[0]):
    indexs[True] = 0
    for j in range(ds.sizes[i]):
      for ip in range(lenPat):
        if (len(patterns[ip])!= indexs[ip] and 
            df.iloc[i][j] == patterns[ip][indexs[ip]]):
          indexs[ip] += 1
          if len(patterns[ip]) == indexs[ip]:
            aireP[ip] += 1
  for i in range(len(aireP)):
    aireP[i] = aireP[i]*len(patterns[i])
  return aireP

In [12]:
patterns = algoArea(ds,10)

In [13]:
aire(ds,patterns)

array([ 529,  616,  897, 1424, 1482, 2227,  589, 2784, 2210,  304])

## Question 4 

Nous allons maintenant tester nos algorithmes avec des jeux de données suggérés dans le sujet.

In [14]:
dataset_1 = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/chess.txt", sep=" ",header=None)
del dataset_1[37]
ds1 = DataSet(dataset_1)

# On réalise algoFrequence sur le dataset_1 5 fois.
dsFrequence1 = algoFrequences(ds1, 5)
# On ressort les fréquences 
freq1 = frequences(ds1, dsFrequence1)

# On réalise algoAire sur le dataset_1.
dsArea1 = algoArea(ds1, 5)
# On ressort les valeurs
area1 = aire(ds1, dsArea1)

In [15]:
dataset_2 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/LEVIATHAN.txt", sep=" ",header=None,)
dataset_2 = dataset_2[0].str.split(' ', expand=True)
ds2 = DataSet(dataset_2)

# On réalise algoFrequence sur le dataset_2 5 fois.
dsFrequence2 = algoFrequences(ds2, 5)
# On ressort les fréquences 
freq2 = frequences(ds2, dsFrequence2)

# On réalise algoAire sur le dataset_2.
dsArea2 = algoArea(ds2, 5)
# On ressort les valeurs
area2 = aire(ds2, dsArea2)

In [16]:
dataset_3 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/FIFA.txt", sep=" ",header=None,)
dataset_3 = dataset_3[0].str.split(' ', expand=True)
ds3 = DataSet(dataset_3)

# On réalise algoFrequence sur le dataset_3 5 fois.
dsFrequence3 = algoFrequences(ds3, 5)
# On ressort les fréquences 
freq3 = frequences(ds3, dsFrequence3)

# On réalise algoAire sur le dataset_3.
dsArea3 = algoArea(ds3, 5)
# On ressort les valeurs
area3 = aire(ds3, dsArea3)

In [17]:
dataset_4 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/BIBLE.txt", sep=" ",header=None,)
dataset_4 = dataset_4[0].str.split(' ', expand=True)
ds4 = DataSet(dataset_4)

# On réalise algoFrequence sur le dataset_4 5 fois.
dsFrequence4 = algoFrequences(ds4, 5)
# On ressort les fréquences 
freq4 = frequences(ds4, dsFrequence4)

# On réalise algoAire sur le dataset_4.
dsArea4 = algoArea(ds4, 5)
# On ressort les valeurs
area4 = aire(ds4, dsArea4)

## Question 5

### Etude emprique

Nous allons étudier les dataset ci-dessus.

#### Dataset n°1

##### Algorithme de fréquence

Dans un premier temps, nous allons executer l'algorithme de la fréquence et chercher à obtenir 5 échantillons.

In [18]:
print("Nous avons extrait les motifs suivants:")
for dsF in dsFrequence1:
    print(dsF)
for f in freq1:
    print("La fréquence est de ", round(f*100,2), "%.")

Nous avons extrait les motifs suivants:
[ 3  5  7  9 12 14 15 18 20 22 23 25 28 29 31 34 36 38 40 42 44 46 48 50
 52 55 56 58 60 62 64 66 68 70 72 74]
[ 3 11 14 15 23 25 29 38 42 46 52 55 60 70 72]
[ 1  3  7  9 11 13 15 17 20 23 25 29 31 36 40 42 45 52 55 58 60 63 66 69
 72 75]
[ 7  9 14 21 25 36 39 48 51 54 56 58 60 62 66 74]
[ 2  7 12 13 18 20 22 24 27 29 34 38 40 44 47 50 52 54 56 58 60 62 64 68
 71 72 74]
La fréquence est de  0.03 %.
La fréquence est de  2.22 %.
La fréquence est de  0.09 %.
La fréquence est de  2.44 %.
La fréquence est de  0.13 %.


*add comments here*

##### Algorithme de l'aire

Dans un second temps, nous allons analyser l'algorithme de l'aire.

In [19]:
print("Nous avons extrait les motifs suivants:")
for dsA in dsArea1:
    print(dsA)
surface = dataset_1.shape
s = surface[0]*surface[1]
print("\nL'aire totale du dataset est de :", s, "\n")
for ar in area1: 
    print("L'aire des motifs précédants est de:", ar, " soit ", round((ar/s)*100, 2), "% du dataset.")

Nous avons extrait les motifs suivants:
[ 3  5  9 11 13 15 19 23 27 31 34 48 52 56 70 72]
[ 3 11 13 17 19 21 23 25 27 36 40 46 48 52 60 62 70 72]
[ 1  3  5  7 11 15 21 27 34 36 38 40 44 46 54 60 64 70 72 74]
[ 1  5 19 27 29 31 34 36 42 46 50 58 72]
[ 3  7  9 13 21 23 25 29 31 34 38 42 44 52 58 62 66 74]

L'aire totale du dataset est de : 118252 

L'aire des motifs précédants est de: 1600  soit  1.35 % du dataset.
L'aire des motifs précédants est de: 756  soit  0.64 % du dataset.
L'aire des motifs précédants est de: 180  soit  0.15 % du dataset.
L'aire des motifs précédants est de: 3354  soit  2.84 % du dataset.
L'aire des motifs précédants est de: 2052  soit  1.74 % du dataset.


*Add some comments here*

#### Dataset n°2

##### Algorithme de fréquence 

Comme pour le dataset précédant, nous allons dans un premier temps executé l'algorithme de la fréquence et obtenir 5 échantillons. 

In [20]:
print("Nous avons extrait les motifs suivants:")
for dsF in dsFrequence2:
    print(dsF)
for f in freq2:
    print("La fréquence est de ", round(f*100,2), "%.")

Nous avons extrait les motifs suivants:
['1' '1' '-1' '1' '1' '-1' '1' '-1' '1' '1' '1' '-1' '1' '-1' '1' '-1' '1'
 '-1' '1' '-1' '1' '1' '-1' '1' '-1' '1' '-1' '1' '-1' '1' '-1' '1' '-1'
 '1' '-1' '1' '-1' '1' '-1' '1' '-1' '1' '-1' '1' '1' '-1' '1' '-1' '1'
 '-1' '365' '-1' '2369' '-1' '-1' '1' '-1' '1' '-1' '1' '-1' '-1' '1' '-1'
 '1' '-1' '1' '-1' '1' '-1' '1' '-1' '1' '-1' '363' '-1' '-1' '2370' '-1'
 '1921' '-1' '-1' '2371' '-1' '-1' '1' '-1' '1' '-1' '1' '-1' '-1' '1' '1'
 '-1' '1' '-1' '1' '-1' '1' '-1' '-1' '1' '-1' '1' '-1' '1' '1' '1' '-1'
 '1' '-1' '1' '-1' '1' '-1' '-1' '1' '-1' '1644' '-1' '2372' '2373' '2374'
 '-1' '1' '-1' '1' '-1' '2375' '-1' '-1' '21' '54' '-1' '989' '-1' '18'
 '-1' '2376' '-1' '17' '-1' '-1' '29' '-1' '17' '-1']
['-1' '-1' '1' '1' '1' '-1' '-1' '-1' '-1' '1' '365' '-1' '1' '1' '-1'
 '2371' '-1' '-1' '1' '1' '-1' '-1' '-1' '1' '1' '-1' '1' '942' '-1']
['1' '-1' '1' '-1' '-1' '1' '-1' '1' '1' '1' '1' '-1' '1' '-1' '1' '1'
 '-1' '-1' '-1' '1' '-1' '1' '

Add some comments here

##### Algorithme de l'aire
Dans un second temps, nous allons exucter l'algorithme de la fréquence et chercher à obtenir un échantillon.

In [21]:
print("Nous avons extrait les motifs suivants:")
for dsA in dsArea2:
    print(dsA)
surface = dataset_2.shape
s = surface[0]*surface[1]
print("\nL'aire totale du dataset est de :", s, "\n")
for ar in area2: 
    print("L'aire des motifs précédants est de:", ar, " soit ", round((ar/s)*100, 2), "% du dataset.")

Nous avons extrait les motifs suivants:
['310' '-1' '103' '4737' '17' '-1' '71' '290' '-1' '5710' '-1' '265' '17'
 '567' '-1' '22' '-1' '8753' '-2']
['1' '1' '-1' '1151' '-1' '-1' '-1' '8' '18' '-1' '193' '-1' '-1' '26'
 '-1']
['36' '37' '38' '-1' '-1' '-1' '40' '-1' '8' '43' '44' '-1' '47' '-1' '49'
 '50' '-1' '51' '52' '-1' '53' '29' '54' '-1' '29' '57' '-1' '58' '3' '59'
 '-1' '-1' '-1' '20' '-1' '61' '-1' '40' '62' '63']
['693' '-1' '685' '-1' '122' '4385' '262' '-1' '-1' '4514' '-1' '265' '-1'
 '1769' '-1' '-1' '685' '-1' '8744' '-1']
['-1' '-1' '71' '-1' '272' '44' '-1' '197' '-1' '-1' '-1' '1285' '279'
 '-1']

L'aire totale du dataset est de : 1085124 

L'aire des motifs précédants est de: 19  soit  0.0 % du dataset.
L'aire des motifs précédants est de: 15  soit  0.0 % du dataset.
L'aire des motifs précédants est de: 40  soit  0.0 % du dataset.
L'aire des motifs précédants est de: 20  soit  0.0 % du dataset.
L'aire des motifs précédants est de: 14  soit  0.0 % du dataset.


Add some comments here

#### Dataset n°3

##### Algorithme de fréquence 

Comme pour le dataset précédant, nous allons dans un premier temps executé l'algorithme de la fréquence et obtenir 5 échantillons. 

In [22]:
print("Nous avons extrait les motifs suivants:")
for dsF in dsFrequence3:
    print(dsF)
for f in freq3:
    print("La fréquence est de ", round(f*100,2), "%.")

Nous avons extrait les motifs suivants:
['17' '33' '155' '13' '70' '117' '-1' '-1' '-1' '-1' '90' '-1' '67' '-1'
 '96' '21' '-1' '-1' '97' '2' '-1' '155' '-1' '21' '-1' '44' '50' '-1'
 '14' '10' '117' '-1' '57' '68' '59' '-1' '70' '-1' '98' '181' '-1' '97'
 '-1' '-1']
['-1' '135' '-1' '21' '59' '-1' '-1' '-1' '33' '97' '135' '-1' '147' '82'
 '96' '-1' '8' '10' '-1' '14' '181' '-1' '98']
['-1' '46' '-1' '48' '47' '-1' '45' '-1' '-1' '32' '-1' '155' '-1' '147'
 '135' '-1' '-1' '131' '-1' '13' '-1' '90' '44' '-1' '50' '-1' '8' '-1'
 '57' '96' '10' '-1' '21' '-1' '24' '-1' '-1' '37' '-1' '30' '-1' '-1'
 '36' '67' '-1' '68' '-1' '70' '97' '-1' '181' '-1' '-1' '98' '-1' '59'
 '117' '-1' '197' '-1' '-1' '138' '221' '-1' '100' '-1' '102' '-1' '297'
 '-1' '-1' '155' '147' '-1' '135' '-1' '131' '-1' '90' '-1' '-1' '2' '-1'
 '13' '-1' '-1' '18' '10' '-1' '24' '-1' '31' '-1' '67' '117' '36' '-1'
 '-1' '82' '-1' '57' '-1' '68' '-1' '59' '-1' '70' '98' '-1' '37' '-1'
 '-1' '181' '-1' '118' '97' '-1'

Add some comments here

##### Algorithme de l'aire
Dans un second temps, nous allons exucter l'algorithme de la fréquence et chercher à obtenir un échantillon.

In [23]:
print("Nous avons extrait les motifs suivants:")
for dsA in dsArea3:
    print(dsA)
surface = dataset_3.shape
s = surface[0]*surface[1]
print("\nL'aire totale du dataset est de :", s, "\n")
for ar in area3: 
    print("L'aire des motifs précédants est de:", ar, " soit ", round((ar/s)*100, 2), "% du dataset.")

Nous avons extrait les motifs suivants:
['142' '-1' '202' '-1' '295' '-1' '-1' '264' '-1' '-1' '-1' '-2']
['147' '-1' '-1' '8' '-1' '-1' '-1' '50' '131' '-1' '18' '-1' '-1' '-2']
['1559' '-1' '1570' '1573' '-1' '1744' '1753' '-1' '680' '-1' '818' '-1'
 '-2']
['17' '-1' '-1' '46' '-1' '-1' '32' '155' '-1' '135' '-1' '-1' '-1' '-2']
['-1' '99' '-1' '124' '-1' '73' '-1' '2503' '42' '-1' '-1' '1001' '-1'
 '162' '-1' '1233' '-1' '-1' '-2']

L'aire totale du dataset est de : 3578750 

L'aire des motifs précédants est de: 3816  soit  0.11 % du dataset.
L'aire des motifs précédants est de: 700  soit  0.02 % du dataset.
L'aire des motifs précédants est de: 13  soit  0.0 % du dataset.
L'aire des motifs précédants est de: 10584  soit  0.3 % du dataset.
L'aire des motifs précédants est de: 19  soit  0.0 % du dataset.


Add some comments here

#### Dataset n°4

##### Algorithme de fréquence 

Comme pour le dataset précédant, nous allons dans un premier temps executé l'algorithme de la fréquence et obtenir 5 échantillons. 

In [24]:
print("Nous avons extrait les motifs suivants:")
for dsF in dsFrequence4:
    print(dsF)
for f in freq4:
    print("La fréquence est de ", round(f*100,2), "%.")

Nous avons extrait les motifs suivants:
['-1' '356' '46' '3659' '-1' '719' '-1' '-1' '-1' '-1' '406' '15' '408'
 '-1' '-1' '2272' '-1' '3' '-1' '55' '-1' '10' '-1' '1064' '-1' '31' '-1'
 '10' '1049' '-1' '22' '-1' '6472' '356' '738' '-1' '-1' '-1' '2272'
 '4119' '154' '-1' '-1' '10' '-1' '-1' '31' '-1' '10' '2272' '-1' '3' '-1'
 '272' '16' '-1' '-1' '356' '-1' '-1' '55' '10' '3' '-1' '-1' '-1' '-1'
 '-1' '-1' '46' '-1' '-1' '6473' '2779' '-1' '-1' '-1' '10' '-1' '-1' '46'
 '-1' '-1' '31' '-1' '55' '10' '12' '-1' '6']
['356' '2373' '-1' '-1' '1602' '-1' '-1' '-1' '-1' '-1' '-1' '10' '-1'
 '365' '-1' '381' '-1' '31' '-1' '46' '-1' '1069' '22' '-1' '-1' '-1' '31'
 '560' '-1' '1503' '59' '-1' '-1' '43' '202' '327' '-1' '-1' '-1' '669'
 '-1' '10' '10' '3004' '4422' '31' '-1' '1895' '-1']
['41' '1283' '15' '1209' '-1' '5714' '1283' '65' '-1' '-1' '-1' '-1' '10'
 '-1' '-1' '-1' '-1' '41' '-1' '46' '-1' '-1' '-1' '-1' '-1' '-1' '-1'
 '-1' '351' '14' '41' '-1' '73' '657' '267' '-1' '-1' '-1' '-

Add some comments here

##### Algorithme de l'aire
Dans un second temps, nous allons exucter l'algorithme de la fréquence et chercher à obtenir un échantillon.

In [25]:
print("Nous avons extrait les motifs suivants:")
for dsA in dsArea4:
    print(dsA)
surface = dataset_4.shape
s = surface[0]*surface[1]
print("\nL'aire totale du dataset est de :", s, "\n")
for ar in area4: 
    print("L'aire des motifs précédants est de:", ar, " soit ", round((ar/s)*100, 2), "% du dataset.")

Nous avons extrait les motifs suivants:
['738' '10' '-1' '456' '-1' '1032' '841' '-1' '10' '-1' '4540' '-1' '-1'
 '46' '-1' '-1' '1213' '-1' '-2']
['2478' '-1' '738' '-1' '-1' '46' '392' '-1' '-1' '10' '-1' '-1' '46'
 '9330']
['-1' '59' '244' '-1' '-1' '59' '-1' '46' '4808' '-2']
['-1' '738' '305' '-1' '4543' '2004' '-1' '13449' '-1' '-1' '4635' '-1'
 '150' '-1' '-1']
['-1' '10' '-1' '-1' '46' '-1' '4699' '10' '-1' '7270' '-1' '-1' '-1']

L'aire totale du dataset est de : 6364575 

L'aire des motifs précédants est de: 19  soit  0.0 % du dataset.
L'aire des motifs précédants est de: 14  soit  0.0 % du dataset.
L'aire des motifs précédants est de: 10  soit  0.0 % du dataset.
L'aire des motifs précédants est de: 15  soit  0.0 % du dataset.
L'aire des motifs précédants est de: 26  soit  0.0 % du dataset.


*Add some comments here*

## Question 6

On a constaté que lorsque l'on avait une ligne beaucoup plus grande que les autres, les algorithmes de fréquence et d'aire favorisent la ligne la plus grande pour créer des motifs. On souhaite trouver des motifs fréquents, donc on souhaite éviter des motifs qui s'appliquent seulement dans des grandes lignes. Nous avons trouver trois solutions:

*   Supprimer les grandes lignes du dataset
*   Réduire la taille des grandes lignes
*   Modifier le poids accorder à la ligne la plus grande



In [26]:
dsetGL = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/FIFA.txt", sep=" ",header=None,)
df_gl= dsetGL[0].str.split(' ', expand=True)
ds_gl = DataSet(df_gl)
ds_gl.df.head()

,0,1,2,3,4,5,6,7,8,9,...,165,166,167,168,169,170,171,172,173,174
0,3,-1,28,-1,58,-1,61,-1,64,-1,...,None,None,None,None,None,None,None,None,None,None
1,4,-1,7,-1,23,-1,255,-1,3,-1,...,None,None,None,None,None,None,None,None,None,None
2,8,-1,10,-1,13,-1,14,-1,21,-1,...,None,None,None,None,None,None,None,None,None,None
3,11,-1,25,-1,40,-1,54,-1,55,-1,...,None,None,None,None,None,None,None,None,None,None
4,12,-1,39,-1,63,-1,72,-1,3,-1,...,None,None,None,None,None,None,None,None,None,None


## Cas où le dataset est chargé dans un df
On va calculer la moyenne du nombre d'item par ligne et en fonction de cela on suprimera les colonnes supérieurs à la taille moyenne du nombre d'item/ligne
On ne regarde pas toute les lignes on en regarde que 40% (on pourras changer)

In [27]:
ds_gl.df.shape

(20450, 175)

In [28]:
def resizeDataSet(ds,percentage):
  df = ds.df
  shape = ds.df.shape
  sum = 0
  #for index, row in df.iterrows():
  for _ in range(int(shape[0]*percentage)):
    #if random.uniform(0.,100.) < 5.:
    sum += shape[1] - ds.sizes[random.randint(0,shape[0]-1)]
  mean = sum/(shape[0]*percentage)
  for i in range(shape[1]-1,int(mean),-1):
    del df[i]
  for i in range(shape[0]):
    if ds.sizes[i] > mean:
        ds.sizes[i] = math.ceil(mean)
  return df

In [29]:
resizeDataSet(ds_gl,0.4)
ds_gl.df.shape

(20450, 103)

## Troisième solution
Text  a mettre richard

Dans cette troisième approche, on va faire une moyenne mobile de la valeur de poid

[Info pour blabla](https://www.educatim.fr/tq/co/Module_TQ_web/co/moyenne_glissante.html)

In [30]:
def algoFrequencesSolutionA(ds,nb_pattern)-> tuple:
  df = ds.df
  sizes = ds.sizes
  R = []
  IsInR = set()
  P: dict = dict()
  w = np.zeros(df.shape[0])
  totalW = 0
    
  # set les probas
  for i in range(len(w)):
    w[i] = math.pow(2,sizes[i])
    if i >= 2:
      w[i] = (w[i] + w[i-1] + w[i-2])/ 3
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    random_row = random.uniform(0,totalW)
    # On cherche la ligne
    row = 0
    v = 0
    for i in range(len(w)):
      if  v > random_row:
        row = i - 1
        break
      v += w[i]
    # On selctionne un motif 
    pattern = np.array(df.iloc[row][:sizes[row]])
    random_v = random.randint(1, len(pattern) - 1 )
    for i in range(len(pattern)- random_v):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    # On ajoute seulement les motifs non présent dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R

In [31]:
patterns = algoFrequencesSolutionA(ds_gl,10)
frequences(ds,patterns)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])